In [2]:
#Donwloading dataset
import requests
import tarfile
import os

output_dir = '../data'
os.makedirs(output_dir, exist_ok=True)

url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

tar_path = os.path.join(output_dir, 'aclImdb_v1.tar.gz')

response = requests.get(url, stream=True)
if response.status_code  == 200:
    with open(tar_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)
    print(f'Downloaded file: {tar_path}')
else:
    print(f'Donwloaded error {response.status_code}')
    
with tarfile.open(tar_path, 'r:gz') as tar_ref:
    tar_ref.extractall(output_dir)
    print(f'Extracted files in: {output_dir}')
    
imdb_dir = os.path.join(output_dir, 'aclImdb')
if os.path.exists(imdb_dir):
    for item in os.listdir(imdb_dir):
        print(f' - {item}')
else:
    print('Error: The folder aclImdb not found')

Downloaded file: ../data\aclImdb_v1.tar.gz
Extracted files in: ../data
 - imdb.vocab
 - imdbEr.txt
 - README
 - test
 - train
